In [22]:
import requests
import json
import os
import foursquare
import pandas as pd
from tqdm import tqdm

# Foursquare

### Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

#### Step 1: Set code to review full API

In [26]:
# Setup Foursquare API credentials 
api_key = os.environ.get('FOURSQUARE_API_KEY')

# Define the endpoint URL for the Foursqare request
endpoint = "https://api.foursquare.com/v3/places/search"

# Define the query parameters to include only the latitude and longitude fields
def foursquare_query(latitude, longitude):
    params = {
        'll': f'{latitude},{longitude}', 'radius':'1000', 'query':'restaurants, coffee shops'
        }
    #Headers specific to FourSquare - used to specify wanted metadata - not typically found under all APIs. Found under https://location.foursquare.com/developer/reference/authentication
    headers = {"Accept": "application/json",
        "Authorization": api_key
        }

    # Send an HTTP GET request to the endpoint URL with the query parameters
    response = requests.get(endpoint, params=params, headers=headers)

    # Get the JSON content from the response
    json_content = response.json()

    # Print out the entire JSON response to see catergory names
    return json_content



#### Step 2: Retrieve Foursquare results

In [27]:
#Retrieve the Foursquare query results for a given latitude and longitude
def bike_radius(latitude, longitude):
    json_content = foursquare_query(latitude, longitude)
    poi = json_content['results']
    return json_content

#extract the relevant information from the Foursquare query result and store it in a dictionary that is returned as a list.
def bike_radius_info(latitude, longitude, location_name):
    json_content = foursquare_query(latitude, longitude)
    poi_list = []
    for poi in json_content['results']:
        poi_dict = {}
        poi_dict['fsq_id'] = poi['fsq_id']
        poi_dict['location_name'] = location_name
        poi_dict['poi_type'] = [category['name'] for category in poi['categories']]
        poi_list.append(poi_dict)
    return poi_list

#### Step 3: Parse through data to retrieve point of interest (POI) details (Available from Foursquare - location, type)

In [30]:
# read the file containing the locations
bike_df = pd.read_csv('/Users/brittanyharding/LHL-Projects/Statistical-Modelling-with-Python/notebooks/London_CityBikes_Data.csv')

rand_df = bike_df.sample(n=250)

# extract the name, latitude, and longitude columns from the City Bike Data
names = rand_df['name'].unique()
latitudes = rand_df['latitude'].unique()
longitudes = rand_df['longitude'].unique()


# create an empty list to store the dictionaries
data_list = []

# loop through each location and call the bike_radius_info function
for i in tqdm(range(len(latitudes))):
    fsq_ids = bike_radius_info(latitudes[i], longitudes[i], names[i])
    for fsq_id in fsq_ids:
        data_list.append({'fsq_id': fsq_id['fsq_id'], 'location_name': fsq_id['location_name'], 'poi_type': fsq_id['poi_type']})

100%|██████████| 250/250 [01:42<00:00,  2.45it/s]


In [29]:
data_list

[{'fsq_id': '5e3575f4dbf5ba00089df22f',
  'location_name': '200091 - Wandsworth Town Station, Wandsworth',
  'poi_type': ['Coffee Shop']},
 {'fsq_id': '515995d5e4b03e8df8ad2e3f',
  'location_name': '200091 - Wandsworth Town Station, Wandsworth',
  'poi_type': ['Coffee Shop']},
 {'fsq_id': '544b84f8498e79ed1befff89',
  'location_name': '200091 - Wandsworth Town Station, Wandsworth',
  'poi_type': ['Café', 'Coffee Shop', 'Restaurant']},
 {'fsq_id': 'a9c747df6e62465ccbd49cd7',
  'location_name': '200091 - Wandsworth Town Station, Wandsworth',
  'poi_type': ['Restaurant']},
 {'fsq_id': '63a0856d6b8c1210f01f4878',
  'location_name': '200091 - Wandsworth Town Station, Wandsworth',
  'poi_type': ['Coffee Shop']},
 {'fsq_id': '616992176d31801e4795a0b0',
  'location_name': '200091 - Wandsworth Town Station, Wandsworth',
  'poi_type': ['Juice Bar']},
 {'fsq_id': '5cbc64b8e55d8b00383630c5',
  'location_name': '200091 - Wandsworth Town Station, Wandsworth',
  'poi_type': ['Coffee Shop']},
 {'fsq_i

#### Step 4:  Put your parsed results into a DataFrame

In [38]:
POI_CityBikes_DF = pd.DataFrame(data_list)

# Yelp

### Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

##### Set code to review full API

In [53]:
# Setup Yelp API credentials 
api_key = os.environ.get('YELP_API_KEY')

# Define the endpoint URL for the Yelp request
endpoint = "https://api.yelp.com/v3/businesses/search"

# Define the query parameters to include only the latitude and longitude fields
def yelp_query(latitude, longitude):
    params = {
        'latitude': latitude , 'longitude': longitude, 'radius': 1000
        }
    #Headers specific to Yelp - used to specify wanted metadata - not typically found under all APIs.
    headers = {'Authorization': f'Bearer {api_key}'}

    # Send an HTTP GET request to the endpoint URL with the query parameters
    response = requests.get(endpoint, params=params, headers=headers)

    # Get the JSON content from the response
    json_content = response.json()

    # Print out the entire JSON response to see catergory names
    return json_content

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [86]:
#Retrieve the Yelp query results for a given latitude and longitude
def bike_radius(latitude, longitude):
    json_content = yelp_query(latitude, longitude)
    poi = json_content['results']
    return json_content

#extract the relevant information from the Yelp query result and store it in a dictionary that is returned as a list.
def bike_radius_info(latitude, longitude, name):
    json_content = yelp_query(latitude, longitude)
    poi_list = []
    for poi in json_content['businesses']:
        poi_dict = {}
        poi_dict['id'] = poi['id']
        poi_dict['location'] = name
        poi_dict['categories'] = poi['categories']
        poi_dict['rating'] = poi['rating']
        poi_dict['name'] = poi['name']
        try:
            poi_dict['rating'] = poi['rating']
        except:
            poi_dict['rating'] = ''
        try:
            poi_dict['price'] = poi['price']
        except:
            poi_dict['price'] = ''
        poi_list.append(poi_dict)
    return poi_list

In [87]:
# read the file containing the locations
bike_df = pd.read_csv('/Users/brittanyharding/LHL-Projects/Statistical-Modelling-with-Python/notebooks/London_CityBikes_Data.csv')

rand_df2 = bike_df.sample(n=5)
# extract the name, latitude, and longitude columns from the City Bike Data
names = rand_df2['name'].unique()
latitudes = rand_df2['latitude'].unique()
longitudes = rand_df2['longitude'].unique()


# create an empty list to store the dictionaries
data_list2 = []

# loop through each location and call the bike_radius_info function
for i in tqdm(range(len(latitudes))):
    id = bike_radius_info(latitudes[i], longitudes[i], names[i])
    for id in id:
        data_list2.append({'id': id['id'], 'name': id['name'], 'location': id['location'], 'categories': id['categories'], 'rating': id['rating'],'price': id['price']})
        

100%|██████████| 5/5 [00:02<00:00,  1.70it/s]


In [88]:
# id = bike_radius_info(latitudes[i], longitudes[i], names[i])

df123 = pd.DataFrame(data_list2)

df123
# # bike_radius('51.529163', '-0.109971')
# latitude = 51.529163
# longitude = -0.109971


# import json

# json_content = bike_radius_info(latitude, longitude, location name)
# print(json.dumps(json_content, indent=4))

,id,name,location,categories,rating,price
0,CexMDiWtQh7BNpq3nSN30g,Prospect Of Whitby,"022166 - Albert Gardens, Stepney","[{'alias': 'pubs', 'title': 'Pubs'}]",4.0,££
1,nUAkhKSEKG3NZl4vinNMjQ,Il Bordello,"022166 - Albert Gardens, Stepney","[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,££
2,xGWqU9FipTH7p-_YKSs9sg,The Narrow by Gordon Ramsay,"022166 - Albert Gardens, Stepney","[{'alias': 'british', 'title': 'British'}, {'a...",3.5,££
3,4VJ5g-TmSUfCVA6Gs7u4lA,Cinnamon Coffee Shop,"022166 - Albert Gardens, Stepney","[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.5,£
4,TKrgGHkG3zZQyHbXLWVGiA,The Grapes,"022166 - Albert Gardens, Stepney","[{'alias': 'pubs', 'title': 'Pubs'}]",4.0,££
...,...,...,...,...,...,...
95,sYwBQ7mJYhB35nn-_SZstQ,Yauatcha,"003424 - Broadcasting House, Marylebone","[{'alias': 'dimsum', 'title': 'Dim Sum'}, {'al...",4.0,£££
96,a8FtrgeFoHZ3N8KPVK2lGQ,Bob Bob Ricard,"003424 - Broadcasting House, Marylebone","[{'alias': 'british', 'title': 'British'}, {'a...",4.0,££££
97,1pUEGW91riUIux0TZWw_4g,Palm Court,"003424 - Broadcasting House, Marylebone","[{'alias': 'tea', 'title': 'Tea Rooms'}]",4.0,£££
98,IBV7BsXnv2gX68nWUARhjw,Mother Mash,"003424 - Broadcasting House, Marylebone","[{'alias': 'british', 'title': 'British'}]",4.0,££


Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating